In [1]:
import matplotlib
import numpy as np
import pandas as pd
from collections import namedtuple
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

EpisodeStats = namedtuple("Stats",["episode_lengths", "episode_rewards"])

def plot_cost_to_go_mountain_car(env, estimator, num_tiles=20):
    x = np.linspace(env.observation_space.low[0], env.observation_space.high[0], num=num_tiles)
    y = np.linspace(env.observation_space.low[1], env.observation_space.high[1], num=num_tiles)
    X, Y = np.meshgrid(x, y)
    Z = np.apply_along_axis(lambda _: -np.max(estimator.predict(_)), 2, np.dstack([X, Y]))

    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                           cmap=matplotlib.cm.coolwarm, vmin=-1.0, vmax=1.0)
    ax.set_xlabel('Position')
    ax.set_ylabel('Velocity')
    ax.set_zlabel('Value')
    ax.set_title("Mountain \"Cost To Go\" Function")
    fig.colorbar(surf)
    plt.show()

# 1
`V`는 키가 상태이고 값이 수확입니다. `V.keys()`는 상태 리스트이고, 상태의 첫번째 항목 `k[0]`은 "현재 선수가 가진 카드의 합"입니다. 상태의 두번째 항목 `k[1]`은 "딜러가 공개한 카드"입니다. `min_x`와 `max_x`는 선수가 가진 카드 합의 최솟값과 최댓값이고, `min_y`와 `max_y`는 딜러가 공개한 카드의 최솟값과 최댓값입니다. `np.arange(최소, 최대)`는 최솟값 부터 최댓값까지 순열을 만듭니다. 순열의 범위가 최소 ≦ < 최대이므로 최댓값에 1을 더합니다.

`np.meshgrid()`는 X 범위와 Y 범위로 만들 수 있는 모든 좌표 조합을 만들고, 각 좌표의 X항 배열과 Y항 배열을 반환합니다. 즉, `np.meshgrid([1, 2, 3], [a, b, c])`의 첫번째 결과는 [[1, 2, 3], [1, 2, 3], [1, 2, 3]]이고 두번째 결과는 [[a, a, a], [b, b, b], [c, c, c]]입니다. `np.meshgrid`의 결과는 그래프를 그리는 함수의 입력으로 사용합니다. `np.dstack`는 여러 행렬의 원소들을 겹쳐서 한 행렬을 만듭니다. 예를 들어, `np.dstack([X, Y])`의 결과는 모든 X, Y 좌표를 조합한 [ [[1,a], [2,a], [3,a]], [[1,b], [2,b], [3,b]], [[1,c], [2,c], [3,c]] ]입니다.

# 2

`np.apply_along_axis`은 축(2, 즉 3번째 차원) 방향으로 모든 향목에 대해 첫번째 아규먼트 함수를 실행합니다. 여기서는 모든 X 범위와 Y 범위에 대해 [X좌표,Y좌표]를 입력으로 함수를 실행합니다. 그래서 `V[(_[0], _[1], False)]`는 `V[(X좌표, Y좌표, False)]`가 됩니다. X좌표는 선수의 카드 합이고 Y좌표는 딜러가 공개한 카드이기 때문에 `Z_noace`는 유저블 에이스를 보유하지 않은 경우 전체 수확 표를 저장합니다. 반대로 `Z_ace`는 `V[(_[0], _[1], True)]`에서 `True` 때문에 유저블 에이스를 보유한 경우 수확을 저장합니다. 의미있는 변수 이름으로 `_`를 사용하는 것은 바람직하지 않습니다.

# 3

그래프를 두개씩 그리므로 공통되는 부분을 함수로 만들었습니다. 그래프 크기, 시점, 면 색깔, 축 이름, 제목 등을 정하는 함수는 Matplotlib 문서를 참고하세요.

In [2]:
def plot_value_function(V, title="Value Function"):
    """
    Plots the value function as a surface plot.
    """
    ###< 1 >###
    min_x = min(k[0] for k in V.keys())
    max_x = max(k[0] for k in V.keys())
    min_y = min(k[1] for k in V.keys())
    max_y = max(k[1] for k in V.keys())

    x_range = np.arange(min_x, max_x + 1)
    y_range = np.arange(min_y, max_y + 1)
    X, Y = np.meshgrid(x_range, y_range)

    ###< 2 >###
    # Find value for all (x, y) coordinates
    Z_noace = np.apply_along_axis(lambda _: V[(_[0], _[1], False)], 2, np.dstack([X, Y]))
    Z_ace = np.apply_along_axis(lambda _: V[(_[0], _[1], True)], 2, np.dstack([X, Y]))

    ###< 3 >###
    def plot_surface(X, Y, Z, title):
        fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(111, projection='3d')
        surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                               cmap=matplotlib.cm.coolwarm, vmin=-1.0, vmax=1.0)
        ax.set_xlabel('Player Sum')
        ax.set_ylabel('Dealer Showing')
        ax.set_zlabel('Value')
        ax.set_title(title)
        ax.view_init(ax.elev, -120)
        fig.colorbar(surf)
        plt.show()

    plot_surface(X, Y, Z_noace, "{} (No Usable Ace)".format(title))
    plot_surface(X, Y, Z_ace, "{} (Usable Ace)".format(title))

In [3]:
def plot_episode_stats(stats, smoothing_window=10, noshow=False):
    # Plot the episode length over time
    fig1 = plt.figure(figsize=(10,5))
    plt.plot(stats.episode_lengths)
    plt.xlabel("Episode")
    plt.ylabel("Episode Length")
    plt.title("Episode Length over Time")
    if noshow:
        plt.close(fig1)
    else:
        plt.show(fig1)

    # Plot the episode reward over time
    fig2 = plt.figure(figsize=(10,5))
    rewards_smoothed = pd.Series(stats.episode_rewards).rolling(smoothing_window, min_periods=smoothing_window).mean()
    plt.plot(rewards_smoothed)
    plt.xlabel("Episode")
    plt.ylabel("Episode Reward (Smoothed)")
    plt.title("Episode Reward over Time (Smoothed over window size {})".format(smoothing_window))
    if noshow:
        plt.close(fig2)
    else:
        plt.show(fig2)

    # Plot time steps and episode number
    fig3 = plt.figure(figsize=(10,5))
    plt.plot(np.cumsum(stats.episode_lengths), np.arange(len(stats.episode_lengths)))
    plt.xlabel("Time Steps")
    plt.ylabel("Episode")
    plt.title("Episode per time step")
    if noshow:
        plt.close(fig3)
    else:
        plt.show(fig3)

    return fig1, fig2, fig3